In [1]:
def infoGain(P,N):
    import math
    p = P/(P+N)
    n = N/(P+N)
    if p==0:
        p=1
    if n==0:
        n=1
    return -p*math.log2(p)-n*math.log2(n)

In [2]:
def getpath(nested_dict, value, prepath=0):
    for k, v in nested_dict.items():
        path = prepath
        if v == value:
            return path+1
        elif hasattr(v, 'items'): 
            p = getpath(v, value, path) 
            if p is not None:
                return p
    return prepath

In [3]:
def finditem(obj, key):
    for k, v in obj.items():
        if v == key:
            return k
        elif hasattr(v, 'items'): 
            p = finditem(v, key) 
            if p is not None:
                return p

In [4]:
def insertNode(tree,Node,AttPosNeg):
    for k, v in tree.items():
        if v == 'None':
            NV = {}
            for i in AttPosNeg:
                for j in AttPosNeg[i]:
                    NV[j]='None'
                    if AttPosNeg[i][j] == 0:
                        NV[j]='No'
                    if AttPosNeg[i][j] == 0:
                        NV[j]='Yes'
            tree[k]={Node:NV}
        elif hasattr(v, 'items'):
            p = insertNode(v, Node,AttPosNeg)
            if p is not 'None':
                return p
    return tree

In [5]:
def insertTree(tree,Node,AttPosNeg):
    for k, v in tree.items():
        if v == 'None':
            tree[k]={k:NV}
        elif hasattr(v, 'items'):
            p = insertNode(v, Node,AttPosNeg)
            if p is not 'None':
                return p
    return tree

In [6]:
def countConcept(data,attr,val):
    from collections import Counter
    cnt=Counter(x for x in data[attr])
    return cnt[val]

In [7]:
def countAttr(data,attr,val,concept,cVal):
    count={}
    C = data[concept]
    A = data[attr]
    for a in data.index:
        for v in val:
            if v not in count:
                count[v] = 0
            if(A[a]==v and C[a]==cVal):
                if v not in count:
                    count[v]=1
                else:
                    count[v]=count[v]+1
    return count

In [8]:
def findItemsList(obj, key,prepath=[]):
    for k, v in obj.items():
        path = prepath
        if v == key:
            path.append(k)
        elif hasattr(v, 'items'): 
            p = findItemsList(v, key, path) 
            if p is not []:
                return p
    return prepath

In [9]:
def getNextData(data,Node,d):
    iData = (data[Node] == d)
    nD = data[iData]
    nD = nD.drop(Node,axis=1)
    return nD

In [28]:
def getTree(data,conceptVals,tree):
    concept=str(list(data)[-1])
    AttributeList=list(data)[:-1]
    Total=data.shape[0]
    pathToNone = getpath(tree,'None')
    if(pathToNone>0 and Total>0):
        Attr = {}
        for a in AttributeList:
            Attr[a] = list(set(data[a]))
        
        AttrCount = {}
        countC={}
        for val in Attr:
            AttrCount[val] = {}
            for cVal in conceptVals:
                countC[cVal] = 0
                countC[cVal] = countConcept(data,concept,cVal)
            for cVal in conceptVals:
                AttrCount[val][cVal] = countAttr(data,val,Attr[val],concept,cVal)
#         print(AttrCount)
#         print(countC)
        EntropyAttr = {}
        AttPosNeg = {}
        ClassEntropy = infoGain(countC['Yes'],countC['No'])
#         print("Class Entropy : ",ClassEntropy)
        if ClassEntropy == 0:
            return tree
        for att in AttrCount:
            InfoGain = {}
            ap = {}
            an = {}
            for c in conceptVals:
                for val in AttrCount[att][c]:
                    if c == 'No':
                        an[val] = AttrCount[att][c][val]
                    if c == 'Yes':
                        ap[val] = AttrCount[att][c][val]
            AttPosNeg[att]={}
            AttPosNeg[att] = {'Yes':ap,'No':an}
            TotalInfo = {}
            for val in AttrCount[att][c]:
                TotalInfo[val] = ap[val]+an[val]
                if ap[val]==0 or an[val]==0:
                    InfoGain[val]=0
                else:
                    InfoGain[val] = infoGain(ap[val],an[val])
#             print("Information Gain : ",InfoGain)
            for val in InfoGain:
                if att not in EntropyAttr:
                    EntropyAttr[att] = ( TotalInfo[val] / Total ) * InfoGain[val]
                else:
                    EntropyAttr[att] = EntropyAttr[att] + ( TotalInfo[val] / Total ) * InfoGain[val]
#         print("Attribute Entropy : ",EntropyAttr)
        Gain = {}
        for g in EntropyAttr:
            if g not in Gain:
                Gain[g] = ClassEntropy - EntropyAttr[g]
#         print("Gain : ",Gain)
        Node = max(Gain,key=Gain.get)
        print("Next root : ",Node)
        tree = insertNode(tree,Node,AttPosNeg[Node])
        print(tree)
        nDL = list(set(findItemsList(tree,'None')))
        for d in nDL:
            newData = getNextData(data,Node,d)
            tree1 = getTree(newData,conceptVals,tree)
            tree = insertTree(tree,tree1,AttPosNeg[Node]) 
        return tree
    else:
        return tree

In [29]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    from collections import Counter
    data = DataFrame.from_csv('play.csv')
    concept=str(list(data)[-1])
    conceptVals = list(set(data[concept]))
    tree = getTree(data,conceptVals,{'root':'None'})
    print(tree)

In [30]:
main()

Next root :  Outlook
{'root': {'Outlook': {'Rain': 'None', 'Overcast': 'Yes', 'Sunny': 'None'}}}


TypeError: unhashable type: 'dict'